Setup environment

In [1]:
!kgtk

usage: kgtk [options] command [ / command]*


In [2]:
# Parameters

# Folder on local machine where to create the output and temporary folders
output_path = "/Users/grantxie/Downloads/NIH"

# The names of the output and temporary folders
output_folder = "useful_wikidata_files"
temp_folder = "temp.useful_wikidata_files"

# The location of input files
wiki_root_folder = "/Users/grantxie/Downloads/NIH/"
claims_file = "claims.tsv.gz"
label_file = "labels.en.tsv.gz"
alias_file = "aliases.en.tsv.gz"
description_file = "descriptions.en.tsv.gz"
item_file = "claims.wikibase-item.tsv.gz"

label_all = "labels.tsv.gz"
alias_all = "aliases.tsv.gz"
description_all = "descriptions.tsv.gz"

# Location of the cache database for kypher
cache_path = "/Users/grantxie/Downloads/NIH/temp.useful_wikidata_files"

# Whether to delete the cache database
delete_database = False

# Whether to compute pagerank as it may not run on the laptop
compute_pagerank = True
languages = 'ru,es,zh-cn,de,it,nl,pl,fr,pt,sv'

In [3]:
languages = languages.split(',')

In [34]:
import io
import os
import subprocess
import sys
import glob

import numpy as np
import pandas as pd

import altair as alt

## Set up environment and folders to store the files

- `OUT` folder where the output files go
- `TEMP` folder to keep temporary files , including the database
- `kgtk` shortcut to invoke the kgtk software
- `kypher` shortcut to invoke `kgtk query with the cache database
- `CLAIMS` the `all.tsv` file of wikidata that contains all edges except label/alias/description
- `LABELS` the file with the English labels
- `ITEMS` the wikibase-item file (currently does not include node1 that are properties so for now we need the net file
- `STORE` location of the cache file

In [5]:
if cache_path:
    os.environ['STORE'] = "{}/wikidata.sqlite3.db".format(cache_path)
else:
    os.environ['STORE'] = "{}/{}/wikidata.sqlite3.db".format(output_path, temp_folder)
os.environ['OUT'] = "{}/{}".format(output_path, output_folder)
os.environ['TEMP'] = "{}/{}".format(output_path, temp_folder)
os.environ['kgtk'] = "kgtk"
os.environ['kgtk'] = "kgtk --debug"
os.environ['kypher'] = "kgtk --debug query --graph-cache " + os.environ['STORE']
os.environ['CLAIMS'] = wiki_root_folder + claims_file
os.environ['LABELS'] = wiki_root_folder + label_file
os.environ['ALIASES'] = wiki_root_folder + alias_file
os.environ['DESCRIPTIONS'] = wiki_root_folder + description_file
os.environ['ITEMS'] = wiki_root_folder + item_file

Echo the variables to see if they are all set correctly

In [6]:
!echo $OUT
!echo $TEMP
!echo $kgtk
!echo $kypher
!echo $CLAIMS
!echo $LABELS
!echo $ALIASES
!echo $LABELS
!echo $DESCRIPTIONSa
!echo $STORE
!alias col="column -t -s $'\t' "

/Users/grantxie/Downloads/NIH/useful_wikidata_files
/Users/grantxie/Downloads/NIH/temp.useful_wikidata_files
kgtk --debug
kgtk --debug query --graph-cache /Users/grantxie/Downloads/NIH/temp.useful_wikidata_files/wikidata.sqlite3.db
/Users/grantxie/Downloads/NIH/claims.tsv.gz
/Users/grantxie/Downloads/NIH/labels.en.tsv.gz
/Users/grantxie/Downloads/NIH/aliases.en.tsv.gz
/Users/grantxie/Downloads/NIH/labels.en.tsv.gz

/Users/grantxie/Downloads/NIH/temp.useful_wikidata_files/wikidata.sqlite3.db


Go to the output directory and create the subfolders for the output files and the temporary files

In [7]:
cd $output_path

/Users/grantxie/Downloads/NIH


In [8]:
!mkdir -p $OUT
!mkdir -p $TEMP

Clean up the output and temp folders before we start

In [9]:
# !rm $OUT/*.tsv $OUT/*.tsv.gz
# !rm $TEMP/*.tsv $TEMP/*.tsv.gz

In [10]:
if delete_database:
    print("Deleteddatabase") 
    !rm $STORE

In [11]:
!ls -l $OUT
!ls $TEMP
!ls -l "$CLAIMS"
!ls -l "$LABELS"
!ls -l "$ALIASES"
!ls -l "$LABELS"
!ls -l "$DESCRIPTIONS"
!ls $STORE

total 282840
-rw-r--r--  1 grantxie  staff      9025 Jul 10 14:52 Untitled.ipynb
-rw-r--r--@ 1 grantxie  staff   4125423 Jul  9 11:27 coinvestigators 2.tsv
-rw-r--r--@ 1 grantxie  staff   4125423 Jul  9 11:56 coinvestigators 3.tsv
-rw-r--r--@ 1 grantxie  staff   4125423 Jul  9 11:58 coinvestigators 4.tsv
-rw-r--r--@ 1 grantxie  staff   2511815 Jul  9 12:26 coinvestigators.compact 2.tsv
-rw-r--r--@ 1 grantxie  staff   2511815 Jul  9 11:23 coinvestigators.compact.tsv
-rw-r--r--@ 1 grantxie  staff    780726 Dec 14 01:16 coinvestigators.compact.tsv.gz
-rw-r--r--@ 1 grantxie  staff   4125423 Jul  9 11:24 coinvestigators.tsv
-rw-r--r--@ 1 grantxie  staff    891571 Dec 14 01:15 coinvestigators.tsv.gz
-rw-r--r--  1 grantxie  staff      4378 Jul  8 20:46 coinvestigators11.tsv.gz
-rw-r--r--@ 1 grantxie  staff     92690 Jul  8 21:51 nih_investigators_for_tl 2.tsv
-rw-r--r--@ 1 grantxie  staff     92690 Jul  8 21:51 nih_investigators_for_tl 3.tsv
-rw-r--r--@ 1 grantxie  staff     27653 Jul  8 20:4

## Concatenate and load KGTK NIH Reporter files

In [63]:
person = ''
inv = ''
org = ''
for file in glob.glob('/Users/grantxie/Downloads/kgtk-files-nih-V3.0_new/*.tsv'):
    if 'person' in file:
        person += ' -i ' + file
    if 'nih_inv' in file:
        inv += ' -i ' + file
    if 'person' in file:
        org += ' -i ' + file

In [47]:
!kgtk cat $person -o person_temp.tsv

In [53]:
df = pd.read_csv('person_temp.tsv', sep = '\t', error_bad_lines=False, warn_bad_lines=False)
df.to_csv('person_temp.tsv', index = False, sep = '\t')

In [54]:
!kgtk add_id -i person_temp.tsv -o /Users/grantxie/Downloads/kgtk-files-nih-V3.0_new/nih_person.id.tsv

In [64]:
!kgtk cat $inv -o inv_temp.tsv

In [65]:
df = pd.read_csv('inv_temp.tsv', sep = '\t', error_bad_lines=False, warn_bad_lines=False)
df.to_csv('inv_temp.tsv', index = False, sep = '\t')

In [66]:
!kgtk add_id -i inv_temp.tsv -o /Users/grantxie/Downloads/kgtk-files-nih-V3.0_new/nih_investigator.id.tsv

In [71]:
!kgtk cat $org -o org_temp.tsv

In [72]:
df = pd.read_csv('org_temp.tsv', sep = '\t', error_bad_lines=False, warn_bad_lines=False)
df.to_csv('org_temp.tsv', index = False, sep = '\t')

In [73]:
!kgtk add_id -i org_temp.tsv -o /Users/grantxie/Downloads/kgtk-files-nih-V3.0_new/nih_organization.id.tsv

In [70]:
pro = ''
for file in glob.glob('/Users/grantxie/Downloads/nih_projects/*.tsv'):
    if 'project' in file:
        pro += ' -i ' + file

In [74]:
!kgtk cat $inv -o pro_temp.tsv

In [75]:
df = pd.read_csv('pro_temp.tsv', sep = '\t', error_bad_lines=False, warn_bad_lines=False)
df.to_csv('pro_temp.tsv', index = False, sep = '\t')

In [76]:
!kgtk add_id -i pro_temp.tsv -o /Users/grantxie/Downloads/nih_projects/nih_project.id.tsv

## Setup coinvestigators and output tl files

In [12]:
!$kypher \
-i "/Users/grantxie/Downloads/kgtk-files-nih-V3.0_new/nih_investigator.id.tsv" --as inv \
--limit 5

[2021-12-14 01:13:17 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_32 AS graph_32_c1
     LIMIT ?
  PARAS: [5]
---------------------------------------------
id	node1	label	node2
QNIHPRO9983961-P8329-QNIHPER1860172	QNIHPRO9983961	P8329	QNIHPER1860172
QNIHPRO9950743-P8329-QNIHPER9569267	QNIHPRO9950743	P8329	QNIHPER9569267
QNIHPRO9950743-P8329-QNIHPER10426588	QNIHPRO9950743	P8329	QNIHPER10426588
QNIHPRO10121452-P8329-QNIHPER6081502	QNIHPRO10121452	P8329	QNIHPER6081502
QNIHPRO10121452-P8329-QNIHPER16339995	QNIHPRO10121452	P8329	QNIHPER16339995


In [14]:
!$kypher \
-i "/Users/grantxie/Downloads/kgtk-files-nih-V3.0_new/nih_person.id.tsv" --as  per\
--limit 5

[2021-12-14 01:15:26 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_33 AS graph_33_c1
     LIMIT ?
  PARAS: [5]
---------------------------------------------
id	node1	label	node2
QNIHPER1904041-label-1b2172	QNIHPER1904041	label	ROBERT STERLING TURNER
QNIHPER1904041-PNIHID-475a43	QNIHPER1904041	PNIHID	1904041
QNIHPER1904041-P108-QNIHORG2059802	QNIHPER1904041	P108	QNIHORG2059802
QNIHORG2059802-PNIHID-d70604	QNIHORG2059802	PNIHID	2059802
QNIHPER6240796-label-d0a9cc	QNIHPER6240796	label	RUTH OTTMAN


In [19]:
!$kypher \
-i "/Users/grantxie/Downloads/kgtk-files-nih-V3.0_new/nih_organization.id.tsv" --as organization\
--limit 5

[2021-12-14 01:20:13 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_25 AS graph_25_c1
     LIMIT ?
  PARAS: [5]
---------------------------------------------
id	node1	label	node2
QNIHORG9087701-label-d0888a	QNIHORG9087701	label	"UNIVERSITY OF WASHINGTON"
QNIHstate_wa_united_states-label-37dccd	QNIHstate_wa_united_states	label	"WA"
QNIHORG9087701-PNIHstate-7b257a	QNIHORG9087701	PNIHstate	"QNIHstate_wa_united_states"
QNIHcity_seattle_wa_united_states-label-b12b73	QNIHcity_seattle_wa_united_states	label	"SEATTLE"
QNIHORG9087701-PNIHcity-e52389	QNIHORG9087701	PNIHcity	"QNIHcity_seattle_wa_united_states"


In [15]:
!$kypher -i inv -i per \
--match '\
    inv: (project)-[]->(person1), \
    inv: (project)-[]->(person2), \
    per: (person2)-[:label]->(person2_name)' \
--where 'person1 != person2' \
--return 'distinct person1 as node1, "Pcoinvestigator_names" as label, person2_name as `node2`' \
--order-by 'person1' \
-o "$OUT"/coinvestigators.tsv.gz



[2021-12-14 01:15:47 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_32_c1."node2" "_aLias.node1", ? "_aLias.label", graph_33_c3."node2" "_aLias.node2"
     FROM graph_32 AS graph_32_c1
     INNER JOIN graph_32 AS graph_32_c2, graph_33 AS graph_33_c3
     ON graph_32_c1."node1" = graph_32_c2."node1"
        AND graph_32_c2."node2" = graph_33_c3."node1"
        AND graph_33_c3."label" = ?
        AND (graph_32_c1."node2" != graph_32_c2."node2")
     ORDER BY graph_32_c1."node2" ASC
  PARAS: ['Pcoinvestigator_names', 'label']
---------------------------------------------
[2021-12-14 01:15:47 sqlstore]: CREATE INDEX "graph_33_node1_idx" ON "graph_33" ("node1")
[2021-12-14 01:15:48 sqlstore]: ANALYZE "graph_33_node1_idx"
[2021-12-14 01:15:48 sqlstore]: CREATE INDEX "graph_33_label_idx" ON "graph_33" ("label")
[2021-12-14 01:15:49 sqlstore]: ANALYZE "graph_33_label_idx"


In [16]:
!$kgtk sort --columns node1 -i "$OUT"/coinvestigators.tsv.gz \
/ compact --mode=NONE --columns node1 label --presorted True \
-o "$OUT"/coinvestigators.compact.tsv.gz

In [31]:
!$kypher -i "$OUT"/coinvestigators.compact.tsv.gz -i organization -i per \
--match ' \
    coinvestigators: (person)-[]->(coinvestigator_names), \
    per: (person)-[:P108]->(organization), \
    per: (person)-[:label]->(person_name), \
    organization: (organization)-[]->(organization_name)' \
--return ' \
    person as person, \
    person_name as person_name, \
    organization as organization, \
    organization_name as organization_name, \
    coinvestigator_names as coinvestigator_names' \
-o "$OUT"/nih_investigators_for_tl_original.tsv.gz


[2021-12-14 03:01:33 query]: SQL Translation:
---------------------------------------------
  SELECT graph_66_c1."node1" "_aLias.person", graph_33_c3."node2" "_aLias.person_name", graph_33_c2."node2" "_aLias.organization", graph_25_c4."node2" "_aLias.organization_name", graph_66_c1."node2" "_aLias.coinvestigator_names"
     FROM graph_25 AS graph_25_c4
     INNER JOIN graph_33 AS graph_33_c2, graph_33 AS graph_33_c3, graph_66 AS graph_66_c1
     ON graph_33_c2."node2" = graph_25_c4."node1"
        AND graph_66_c1."node1" = graph_33_c2."node1"
        AND graph_66_c1."node1" = graph_33_c3."node1"
        AND graph_33_c2."label" = ?
        AND graph_33_c3."label" = ?
  PARAS: ['P108', 'label']
---------------------------------------------
^C

Keyboard interrupt in query --graph-cache /Users/grantxie/Downloads/NIH/temp.useful_wikidata_files/wikidata.sqlite3.db -i /Users/grantxie/Downloads/NIH/useful_wikidata_files/coinvestigators.compact.tsv.gz -i organization -i per --match       coinve

In [30]:
!$kypher -i organization \
--match ' \
    organization: (organization)-[:PNIHcity]->(city), \
    organization: (organization)-[:PNIHstate]->(state), \
    organization: (organization)-[:PNIHcountry]->(country), \
    organization: (city)-[:label]->(cname), \
    organization: (state)-[:label]->(sname), \
    organization: (country)-[:label]->(coname), \
    organization: (organization)-[:label]->(oname)' \
--return ' \
    oname as organization,\
    cname as city, \
    sname as state, \
    coname as country' \
-o "$OUT"/nih_investigators_for_tl_original.tsv.gz

[2021-12-14 02:54:52 query]: SQL Translation:
---------------------------------------------
  SELECT graph_25_c7."node2" "_aLias.organization", graph_25_c4."node2" "_aLias.city", graph_25_c5."node2" "_aLias.state", graph_25_c6."node2" "_aLias.country"
     FROM graph_25 AS graph_25_c1
     INNER JOIN graph_25 AS graph_25_c2, graph_25 AS graph_25_c3, graph_25 AS graph_25_c4, graph_25 AS graph_25_c5, graph_25 AS graph_25_c6, graph_25 AS graph_25_c7
     ON graph_25_c1."node1" = graph_25_c2."node1"
        AND graph_25_c1."node1" = graph_25_c3."node1"
        AND graph_25_c1."node1" = graph_25_c7."node1"
        AND graph_25_c1."node2" = graph_25_c4."node1"
        AND graph_25_c2."node2" = graph_25_c5."node1"
        AND graph_25_c3."node2" = graph_25_c6."node1"
        AND graph_25_c1."label" = ?
        AND graph_25_c2."label" = ?
        AND graph_25_c3."label" = ?
        AND graph_25_c4."label" = ?
        AND graph_25_c5."label" = ?
        AND graph_25_c6."label" = ?
        AND g